In [1]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.9/530.9 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 12.7 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np
from huggingface_hub import interpreter_login


In [3]:
import os
# disable Weights and Biases
os.environ['WANDB_DISABLED']="true"


In [4]:
huggingface_dataset_name = "neil-code/dialogsum-test"
dataset = load_dataset(huggingface_dataset_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1999 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/499 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/499 [00:00<?, ? examples/s]

In [5]:
dataset['train'][0]

{'id': 'train_0',
 'dialogue': "#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, thanks doctor.",
 'summary': "Mr. Smith'

In [6]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

In [7]:
from transformers import AutoConfig

model_name='microsoft/phi-2'
device_map = {"": 0}

# Load tokenizer first to get pad_token_id
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

# Load model configuration and set pad_token_id
config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
config.pad_token_id = tokenizer.pad_token_id

original_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                      config=config, # Pass the modified config
                                                      device_map=device_map,
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/453 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

In [9]:
%%time
from transformers import set_seed
seed = 42
set_seed(seed)

def gen(model,p,max_new_tokens=100):
    tokenized_p = tokenizer(p, return_tensors='pt').to(model.device)
    outputs = model.generate(input_ids=tokenized_p.input_ids,
                             attention_mask=tokenized_p.attention_mask,
                             max_new_tokens=max_new_tokens,
                             do_sample=False)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

index = 10

prompt = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

formatted_prompt = f"Instruct: Summarize the following conversation.\n{prompt}\nOutput:\n"
res = gen(original_model,formatted_prompt,100,)
#print(res[0])
output = res[0].split('Output:\n')[1]

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruct: Summarize the following conversation.
#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday
Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# attends Brian's birthday pa

In [10]:
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('instruction','output')
    Then concatenate them using two newline characters
    :param sample: Sample dictionnary
    """
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruct: Summarize the below conversation."
    RESPONSE_KEY = "### Output:"
    END_KEY = "### End"

    blurb = f"\n{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"{sample['dialogue']}" if sample["dialogue"] else None
    response = f"{RESPONSE_KEY}\n{sample['summary']}"
    end = f"{END_KEY}"

    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    formatted_prompt = "\n\n".join(parts)
    sample["text"] = formatted_prompt

    return sample

In [11]:
from functools import partial

def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length


def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )

def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int,seed, dataset):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    """

    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)#, batched=True)

    # Apply preprocessing to each batch of the dataset & and remove 'instruction', 'context', 'response', 'category' fields
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=['id', 'topic', 'dialogue', 'summary'],
    )

    # Filter out samples that have input_ids exceeding max_length
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

    # Shuffle dataset
    dataset = dataset.shuffle(seed=seed)

    return dataset

In [12]:
## Pre-process dataset
max_length = get_max_length(original_model)
print(max_length)

train_dataset = preprocess_dataset(tokenizer, max_length,seed, dataset['train'])
eval_dataset = preprocess_dataset(tokenizer, max_length,seed, dataset['validation'])

Found max lenth: 2048
2048
Preprocessing dataset...


Map:   0%|          | 0/1999 [00:00<?, ? examples/s]

Map:   0%|          | 0/1999 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1999 [00:00<?, ? examples/s]

Preprocessing dataset...


Map:   0%|          | 0/499 [00:00<?, ? examples/s]

Map:   0%|          | 0/499 [00:00<?, ? examples/s]

Filter:   0%|          | 0/499 [00:00<?, ? examples/s]

In [13]:
from peft import prepare_model_for_kbit_training

# 2 - Using the prepare_model_for_kbit_training method from PEFT
# Preparing the Model for QLoRA
original_model = prepare_model_for_kbit_training(original_model)

In [14]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

config = LoraConfig(
    r=32, #Rank
    lora_alpha=32,
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense'
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

# 1 - Enabling gradient checkpointing to reduce memory usage during fine-tuning
original_model.gradient_checkpointing_enable()

peft_model = get_peft_model(original_model, config)

In [15]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 20971520
all model parameters: 1542364160
percentage of trainable model parameters: 1.36%


In [16]:
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'
import transformers

peft_training_args = transformers.TrainingArguments(
    output_dir = output_dir,
    warmup_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=1000,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps=25,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=25,
    eval_steps=25,
    do_eval=True,
    gradient_checkpointing=True,
    report_to="none",
    group_by_length=True,
)

peft_model.config.use_cache = False

peft_trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


In [17]:
peft_trainer.train()

Step,Training Loss
25,1.670815
50,1.190740
75,1.449004
100,1.205755
125,1.439287
150,1.133969
175,1.404353
200,1.148935
225,1.443450
250,1.225222


Step,Training Loss
25,1.670815
50,1.190740
75,1.449004
100,1.205755
125,1.439287
150,1.133969
175,1.404353
200,1.148935
225,1.443450
250,1.225222


TrainOutput(global_step=1000, training_loss=1.2938995628356933, metrics={'train_runtime': 2871.0401, 'train_samples_per_second': 1.393, 'train_steps_per_second': 0.348, 'total_flos': 1.854448505524224e+16, 'train_loss': 1.2938995628356933, 'epoch': 2.0})

In [28]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig

base_model_id = "microsoft/phi-2"

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

# Config
config = AutoConfig.from_pretrained(base_model_id, trust_remote_code=True)
config.pad_token_id = tokenizer.pad_token_id

# Base model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    config=config,
    device_map="auto",
    quantization_config=bnb_config,  # your 4/8-bit config
    trust_remote_code=True
)

Loading weights:   0%|          | 0/453 [00:00<?, ?it/s]

In [22]:
eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

In [29]:
from peft import PeftModel
import os

ft_model = PeftModel.from_pretrained(base_model, os.path.join(output_dir, "checkpoint-1000"),torch_dtype=torch.float16,is_trainable=False)

In [31]:
ft_model.eval()                        # eval mode
ft_model.gradient_checkpointing_disable()  # disable gradient checkpointing
ft_model.config.use_cache = True       # enable KV cache

# Ensure GPU
if next(ft_model.parameters()).device.type != "cuda":
    ft_model = ft_model.to("cuda")

# -----------------------------
# Optional: merge adapter for fastest inference
# -----------------------------
merged_model = ft_model.merge_and_unload()
merged_model.eval()

# -----------------------------
# Save final merged model
# -----------------------------
final_dir = "./phi2-dialogue-merged"
merged_model.save_pretrained(final_dir)
tokenizer.save_pretrained(final_dir)
print(f"Final merged model saved to {final_dir}")

# generation function
def gen(model, prompt, max_new_tokens=50):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=max_new_tokens,
        do_sample=False
    )
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Test generation
set_seed(42)

# Pick a test example
index = 5
dialogue = dataset['test'][index]['dialogue']
human_summary = dataset['test'][index]['summary']

prompt = f"Instruct: Summarize the following conversation.\n{dialogue}\nOutput:\n"

# --- Base/original model  ---
base_output = gen(base_model, prompt, 100)[0].split("Output:\n")[1]

# --- PEFT / merged model ---
peft_output = gen(merged_model, prompt, 100)[0].split("Output:\n")[1]

dash_line = "-"*100
print(dash_line)
print("INPUT PROMPT:\n", prompt)
print(dash_line)
print("BASELINE HUMAN SUMMARY:\n", human_summary)
print(dash_line)
print("BASE MODEL SUMMARY (Zero-shot):\n", base_output)
print(dash_line)
print("PEFT MODEL SUMMARY:\n", peft_output)

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


✅ Final merged model saved to ./phi2-dialogue-merged


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


----------------------------------------------------------------------------------------------------
INPUT PROMPT:
 Instruct: Summarize the following conversation.
#Person1#: You're finally here! What took so long?
#Person2#: I got stuck in traffic again. There was a terrible traffic jam near the Carrefour intersection.
#Person1#: It's always rather congested down there during rush hour. Maybe you should try to find a different route to get home.
#Person2#: I don't think it can be avoided, to be honest.
#Person1#: perhaps it would be better if you started taking public transport system to work.
#Person2#: I think it's something that I'll have to consider. The public transport system is pretty good.
#Person1#: It would be better for the environment, too.
#Person2#: I know. I feel bad about how much my car is adding to the pollution problem in this city.
#Person1#: Taking the subway would be a lot less stressful than driving as well.
#Person2#: The only problem is that I'm going to reall

In [34]:
original_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    config=config, # Pass the already configured config object
    device_map='auto',
    quantization_config=bnb_config,
    trust_remote_code=True
)

Loading weights:   0%|          | 0/453 [00:00<?, ?it/s]

In [49]:
import pandas as pd

dialogues = dataset['test'][0:200]['dialogue']
human_baseline_summaries = dataset['test'][0:200]['summary']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    human_baseline_text_output = human_baseline_summaries[idx]
    prompt = f"Instruct: Summarize the following conversation.\n{dialogue}\nOutput:\n"

    original_model_res = gen(original_model,prompt,100,)
    original_model_text_output = original_model_res[0].split('Output:\n')[1]

    peft_model_res = gen(ft_model,prompt,100,)
    peft_model_output = peft_model_res[0].split('Output:\n')[1]
    print(peft_model_output)
    peft_model_text_output, success, result = peft_model_output.partition('###')

    original_model_summaries.append(original_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'peft_model_summaries'])
df

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Person 1: Ms. Dawson, I need you to take a dictation for me.
Person 2: Yes, sir...
Person 1: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
Person 2: Yes, sir. Go ahead.
Person 1: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Person 1: Ms. Dawson, I need you to take a dictation for me.
Person 2: Yes, sir...
Person 1: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
Person 2: Yes, sir. Go ahead.
Person 1: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Person1 suggests that Person2 consider upgrading their system by adding a painting program for advertising purposes and upgrading their hardware for better performance. Person2 agrees and asks for guidance on how to proceed. Person1 recommends adding a faster processor, more memory, a faster modem, and a CD-ROM drive.



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Person1 suggests that Person2 consider upgrading their system by adding a painting program for advertising purposes and upgrading their hardware for better performance. Person2 agrees and asks for guidance on how to proceed. Person1 recommends adding a faster processor, more memory, a faster modem, and a CD-ROM drive.



,human_baseline_summaries,original_model_summaries,peft_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,"Person 1: Ms. Dawson, I need you to take a dic...","Person 1: Ms. Dawson, I need you to take a dic..."
1,In order to prevent employees from wasting tim...,"Person 1: Ms. Dawson, I need you to take a dic...","Person 1: Ms. Dawson, I need you to take a dic..."
2,Ms. Dawson takes a dictation for #Person1# abo...,"Person 1: Ms. Dawson, I need you to take a dic...","Person 1: Ms. Dawson, I need you to take a dic..."
3,#Person2# arrives late because of traffic jam....,Person1 and Person2 are discussing the traffic...,Person1 and Person2 are discussing the traffic...
4,#Person2# decides to follow #Person1#'s sugges...,Person1 and Person2 are discussing the traffic...,Person1 and Person2 are discussing the traffic...
...,...,...,...
195,#Person1# and #Person2# think John's fallen fo...,Person1 and Person2 discuss John's frequent da...,Person1 and Person2 discuss John's frequent da...
196,#Person1# and #Person2# talk about John's dati...,Person1 and Person2 discuss John's frequent da...,Person1 and Person2 discuss John's frequent da...
197,#Person1# and #Person2# are talking about a co...,Person1 and Person2 discuss John's frequent da...,Person1 and Person2 discuss John's frequent da...
198,#Person1# is giving #Person2# some advice for ...,Person1 suggests that Person2 consider upgradi...,Person1 suggests that Person2 consider upgradi...


In [50]:
import evaluate

rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('PEFT MODEL:')
print(peft_model_results)

print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

ORIGINAL MODEL:
{'rouge1': np.float64(0.2268055757798751), 'rouge2': np.float64(0.06383469762332693), 'rougeL': np.float64(0.17606458649668977), 'rougeLsum': np.float64(0.1842673117080092)}
PEFT MODEL:
{'rouge1': np.float64(0.23945830809142185), 'rouge2': np.float64(0.06779183813925205), 'rougeL': np.float64(0.18544097335493515), 'rougeLsum': np.float64(0.1911613250240366)}
Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL
rouge1: 1.27%
rouge2: 0.40%
rougeL: 0.94%
rougeLsum: 0.69%


In [51]:
bleu = evaluate.load('bleu')

# BLEU expects references as lists of lists of strings
bleu_original = bleu.compute(
    predictions=original_model_summaries,
    references=[[ref] for ref in human_baseline_summaries]
)
bleu_peft = bleu.compute(
    predictions=peft_model_summaries,
    references=[[ref] for ref in human_baseline_summaries]
)

print("BLEU RESULTS")
print("Original:", bleu_original)
print("PEFT:", bleu_peft)

print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL for BLEU metrics")
for key in bleu_peft.keys():
    if isinstance(bleu_peft[key], (float, int, np.number)): # Check if the value is a scalar
        if key in bleu_original and isinstance(bleu_original[key], (float, int, np.number)):
            improvement = bleu_peft[key] - bleu_original[key]
            print(f"{key} improvement: {improvement*100:.2f}%")
    else:
        print(f"Skipping non-scalar metric: {key}")
print("\n")

BLEU RESULTS
Original: {'bleu': 0.027202312412785527, 'precisions': [0.14738261064929228, 0.03831825439063331, 0.016058056048791784, 0.006037795028620716], 'brevity_penalty': 1.0, 'length_ratio': 2.9231611208406303, 'translation_length': 13353, 'reference_length': 4568}
PEFT: {'bleu': 0.028538092020432497, 'precisions': [0.16033653846153847, 0.04283387622149837, 0.0173841059602649, 0.005555555555555556], 'brevity_penalty': 1.0, 'length_ratio': 2.732049036777583, 'translation_length': 12480, 'reference_length': 4568}
Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL for BLEU metrics
bleu improvement: 0.13%
Skipping non-scalar metric: precisions
brevity_penalty improvement: 0.00%
length_ratio improvement: -19.11%
translation_length improvement: -87300.00%
reference_length improvement: 0.00%




In [52]:
meteor = evaluate.load("meteor")

meteor_original = meteor.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries
)
meteor_peft = meteor.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries
)

print("METEOR RESULTS")
print("Original:", meteor_original)
print("PEFT:", meteor_peft)
meteor_improvement = np.array(list(meteor_peft.values())) - np.array(list(meteor_original.values()))
for key, val in zip(meteor_peft.keys(), meteor_improvement):
    print(f"{key} improvement: {val*100:.2f}%")
print("\n")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


METEOR RESULTS
Original: {'meteor': np.float64(0.24540403493224205)}
PEFT: {'meteor': np.float64(0.25779825723091554)}
meteor improvement: 1.24%




In [53]:
!pip install bert_score
bertscore = evaluate.load("bertscore")

bertscore_original = bertscore.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries,
    model_type="bert-base-uncased"
)
bertscore_peft = bertscore.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries,
    model_type="bert-base-uncased"
)

print("BERTScore RESULTS (F1)")
print("Original (F1 mean):", np.mean(bertscore_original['f1']))
print("PEFT (F1 mean):", np.mean(bertscore_peft['f1']))
print(f"Improvement: {(np.mean(bertscore_peft['f1']) - np.mean(bertscore_original['f1']))*100:.2f}%")
print("\n")

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: bert-base-uncased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


BERTScore RESULTS (F1)
Original (F1 mean): 0.53060551404953
PEFT (F1 mean): 0.5438885763287544
Improvement: 1.33%


